In [21]:
import pandas as pd

exel = pd.ExcelFile('crime.xls')
exel.sheet_names

df = exel.parse('Лист1')
df = df.astype(np.float64)


df.head()

,R,AGE,S,ED,EX0,EX1,LF,M,N,NW,U1,U2,W,X
1,79.1,151.0,1.0,91.0,58.0,56.0,510.0,950.0,33.0,301.0,108.0,41.0,394.0,261.0
2,163.5,143.0,0.0,113.0,103.0,95.0,583.0,1012.0,13.0,102.0,96.0,36.0,557.0,194.0
3,57.8,142.0,1.0,89.0,45.0,44.0,533.0,969.0,18.0,219.0,94.0,33.0,318.0,250.0
4,196.9,136.0,0.0,121.0,149.0,141.0,577.0,994.0,157.0,80.0,102.0,39.0,673.0,167.0
5,123.4,141.0,0.0,121.0,109.0,101.0,591.0,985.0,18.0,30.0,91.0,20.0,578.0,174.0



R: Crime rate: # of offenses reported to police per million population 

Age: The number of males of age 14-24 per 1000 population 

S: Indicator variable for Southern states (0 = No, 1 = Yes) 

Ed: Mean # of years of schooling x 10 for persons of age 25 or older 

Ex0: 1960 per capita expenditure on police by state and local government 

Ex1: 1959 per capita expenditure on police by state and local government 

LF: Labor force participation rate per 1000 civilian urban males age 14-24 

M: The number of males per 1000 females 

N: State population size in hundred thousands 

NW: The number of non-whites per 1000 population 

U1: Unemployment rate of urban males per 1000 of age 14-24 

U2: Unemployment rate of urban males per 1000 of age 35-39 

W: Median value of transferable goods and assets or family income in tens of $ 

X: The number of families per 1000 earning below 1/2 the median income 



# Calculate The Influence of _ED_ and _NW_ on _R_

In [49]:
ed = df[['ED']].values
nw = df[['NW']].values
r = df[['R']].values

In [50]:
import numpy as np
X = np.concatenate([ed,nw], axis=1)

from sklearn import preprocessing
X = preprocessing.StandardScaler().fit(X).transform(X)

In [51]:
from sklearn.feature_selection import mutual_info_regression
mutual_info_regression(X, np.squeeze(r))

array([0.        , 0.13223075])

Are NW values being a stronger predictor of R than ED.

In [52]:
import scipy.stats as stats

chisq, p_value = stats.chisquare(r, ed)
chisq, p_value

alpha = 0.05
"independent" if p_value < alpha else 'dependent', p_value

('independent', array([1.28168949e-110]))

In [53]:
import scipy.stats as stats

chisq, p_value = stats.chisquare(r, nw)
chisq, p_value

alpha = 0.05
"independent" if p_value < alpha else 'dependent', p_value

('independent', array([0.]))

# Build Linear Model

In [58]:
from sklearn import linear_model
regr = linear_model.LinearRegression()
regr.fit(X, y)
# The coefficients
print('Coefficients: ', regr.coef_, regr.intercept_)

Coefficients:  [23.62621367 16.95595822] 90.50851063829788


In [62]:
regr.score(X, y) # Returns the coefficient of determination R^2 of the prediction.

0.21378861933660565

In [64]:
import statsmodels.api as sm
from scipy import stats

est = sm.OLS(y, X).fit()
print(est.summary())

                            OLS Regression Results                            
Dep. Variable:                      R   R-squared:                       0.032
Model:                            OLS   Adj. R-squared:                 -0.011
Method:                 Least Squares   F-statistic:                    0.7538
Date:                Fri, 30 Nov 2018   Prob (F-statistic):              0.476
Time:                        16:28:11   Log-Likelihood:                -281.54
No. Observations:                  47   AIC:                             567.1
Df Residuals:                      45   BIC:                             570.8
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1            23.6262     19.290      1.225      0.2